In [2]:
import torch
import torch.nn as nn

In [ ]:
# 레이어를 많이 쌓을 수록 더 성능이 좋다 -> 실험
# 레이어 16개 생성
# 한계는 19 레이어까지 -> 정보를 잃을 수 있음
# LeNet처럼 shorcut이 없음

class VGG16(nn.Module):
    def __init__(self, num_classes=1000) :
        super(VGG16, self).__init__()

            
        self.features = nn.Sequential([
            
            # 비선형화, 특징점 추출, 고수준의 학습
            # 저층 : 저수준, 고층: 고수준 -> 저수준의 특징 누락 방지
            
            # 1층   -   Conv3-64(3커널, 64인풋)
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),

            # 2층   -   Conv3-128
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128,128,kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(kernel_size=2,stride=2),
            
            # 더 많은 Conv쓰는 이유
            # 모델 표현 능력 증가 위해서 -> 더 많은 Conv를 추가하므로서 모델 표현 능력 증가 시키기 위한 구조
            # 특징 복잠성 추가 -> 첫번 특징 찾고 + 두번 조금 더 특징 찾아서 복잡한 특징을 찾기 위함
            
            # 3층   -   Conv3-256
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(kernel_size=2,stride=2),
            
            # 4층   -   Conv3-512
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2,stride=2),
            
            # 5층   -   Conv3-512
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2,stride=2)
        ])
        
        self.avgpool = nn.AdaptiveAvgPool2d(7,7)    # 입력 크기의 차원축소
        # 보통 마지막에 average pooling으로 마무리 -> 다양한 크기의 이미지를 일관된 크기로 출력할 수 있음
        # 마지막에 maxpooling을 사용하면 input image를 동일하게 해야함
        
                
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            
            nn.Linear(4096, num_classes)
        )

    def forward(self,x) :
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        
        return x